# Оценка результатов A/B-теста

## Описание проекта

### Цель и задачи
Необходимо провести оценку результатов A/B-теста:

1. Оценить корректность проведения теста:

   1. Проверить пересечение тестовой аудитории с конкурирующим тестом;
   2. Проверить совпадение времени проведения теста и маркетинговых событий;
   3. Проверить равномерность распределения пользователей по тестовым группам и принадлежность пользователей к обеим группам.


2. Проанализировать результаты теста:

   1. Проверить, достигнуты ли ожидаемые эффекты (метрики улучшились на ожидаемый процент);
   2. Если ожидаемые эффекты достигнуты, то проверить статистическую значимость изменений (сформулировать гипотезы, выбрать статистический критерий и уровень статистической значимости, применить выбранный статистический критерий и оценить результаты).


### Техническое задание

- Название теста: `recommender_system_test`;
- Группы: А — контрольная, B — новая платёжная воронка;
- Дата запуска: 2020-12-07;
- Дата остановки набора новых пользователей: 2020-12-21;
- Дата остановки: 2021-01-04;
- Аудитория: 15% новых пользователей из региона EU;
- Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;
- Ожидаемое количество участников теста: 6000;
- Ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:
  - Конверсия в просмотры карточек товаров — событие `product_page`,
  - Конверсия в просмотры корзины — `product_cart`,
  - Конверсия в покупки — `purchase`.

## Данные

1. Файл `ab_project_marketing_events.csv` — календарь маркетинговых событий на 2020 год:

   - `name` — название маркетингового события;
   - `regions` — регионы, в которых будет проводиться рекламная кампания;
   - `start_dt` — дата начала кампании;
   - `finish_dt` — дата завершения кампании.


2. Файл `final_ab_new_users.csv` — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года:

   - `user_id` — идентификатор пользователя;
   - `first_date` — дата регистрации;
   - `region` — регион пользователя;
   - `device` — устройство, с которого происходила регистрация.


3. Файл `final_ab_events.csv` — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года:

   - `user_id` — идентификатор пользователя;
   - `event_dt` — дата и время покупки;
   - `event_name` — тип события;
   - `details` — дополнительные данные о событии. Например, для покупок (`purchase`) в этом поле хранится стоимость покупки в долларах.


4. `final_ab_participants.csv` — таблица участников тестов:

   - `user_id` — идентификатор пользователя;
   - `ab_test` — название теста;
   - `group` — группа пользователя.

## <a id='content'>План проекта</a>

1. [Подготовка и настройка](#step-0)
2. [Загрузка и изучение данных](#step-1)
3. [Предобработка данных](#step-2)
4. [Оценка корректности проведения A/B-теста](#step-3)
5. [Исследовательский анализ данных](#step-4)
6. [Оценка результатов A/B-тестирования](#step-5)
7. [Общие выводы](#result)

---

## <a id='step-0'>Шаг 0. Подготовка и настройка¶</a>

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import math as mth
from datetime import datetime

from scipy import stats as st
from termcolor import colored # раскраска текста

import matplotlib.pyplot as plt
import seaborn as sns

ModuleNotFoundError: No module named 'termcolor'

In [ ]:
# отключение предупреждений
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# настройки графиков
large = 16; medium = 14; small = 12

params = {'axes.titlesize': large,
          'axes.labelsize': medium,
          'legend.fontsize': medium,
          'figure.titlesize': large,
          'figure.figsize': (10, 6),
          'xtick.labelsize': small,
          'ytick.labelsize': small,
          'axes.grid': True,
          'legend.loc': 'upper left',
          'axes.titlepad': 20.0, # отступ названия графика
          'axes.labelpad': 8.0, # отступ для подписей осей
          'legend.title_fontsize': medium,
         }
plt.rcParams.update(params)
# plt.rcParams.keys() # список параметров и их значения

plt.style.use('seaborn-muted') # единый стиль графиков
sns.set_style('white')

<div align='right'><a href='#content'>↑ В начало проекта ↑</a></div>

---

## <a id='step-1'>Шаг 1. Загрузка и изучение данных</a>

### 1.1. Автоматизация

Так как датасетов у нас несколько, напишем **функцию `about_df`**, которая будет:
- Принимать на вход имя файла;
- Выводить примеры строк, информацию о датасете и числовое описание данных, количество и долю пропусков по каждому столбцу, количество дубликатов и сами дублирующиеся строки;
- Возвращать прочитанный из файла датафрейм, чтобы его можно было сохранить в переменную.

In [ ]:
def about_df(df_name):   
    data = pd.read_csv('/datasets/{}.csv'.format(df_name))
    
    print(colored('Примеры строк:', attrs=['bold']))
    if len(data)>=5:
        display(data.sample(5))
    else:
        display(data)
    
    print(colored('\nИнформация о датасете:', attrs=['bold']))
    display(data.info())
    
    print(colored('\nЧисловое описание данных:', attrs=['bold']))
    display(data.describe())

    print(colored('\nПропуски (количество и доля):', attrs=['bold']))
    display(data.isna().agg(['sum', 'mean']))
    
    # дубликаты
    duplicates = data[data.duplicated()]
    duplicates_cnt = len(duplicates)
    print(colored('\nКоличество дубликатов:', attrs=['bold']), duplicates_cnt)
    if duplicates_cnt > 0:
        display(duplicates_cnt)
    
    return data

### 1.2. Календарь маркетинговых событий на 2020 год

Воспользуемся написанной нами функцией:

In [ ]:
mark_events = about_df('ab_project_marketing_events')

Пропусков и дубликатов нет, но потребуется преобразовать значения `start_dt` и `finish_dt` в даты.

### 1.3. Данные о пользователях, зарегистрировавшихся с 7 по 21 декабря 2020 года

In [ ]:
users = about_df('final_ab_new_users')

Дополнительно посмотрим на регионы:

In [ ]:
# регионы пользователей
users['region'].value_counts()

Пропусков и дубликатов нет, но нужно преобразовать значения столбца `first_date` в даты.

Так же стоит отметить, что нам понадобятся данные только по пользователям из региона EU.

### 1.4. Действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года

In [ ]:
events = about_df('final_ab_events')

Посмотрим, по событиям каких типов есть информация в поле `details`:

In [ ]:
events.loc[~events['details'].isna(), 'event_name'].value_counts()

Детали есть только по покупкам. Посмотрим, по всем ли покупкам есть детали:

In [ ]:
events.loc[events['event_name'] == 'purchase', 'details'].isna().sum()

Итого, дубликатов нет, а пропуски в `details` объясняются тем, что детали есть по событиям только одного типа (`purchase`). При этом по каждой покупке есть детали.

В этом датасете нужно:
- Преобразовать значения столбца `event_dt` в дату и время;
- Добавить столбец с датой без времени.

### 1.5. Таблица участников тестов

In [ ]:
tests = about_df('final_ab_participants')

Видим:
- Пропусков нет;
- Судя по общему количеству и количеству уникальных значений в поле `user_id`, у нас есть пользователи, которые попали либо в несколько тестов, либо в несколько групп в одном тесте, либо и то, и другое. Выясним это на этапе оценки корректности проведения теста.

### Промежуточные выводы

По итогам ознакомления данных обозначили фронт работы по предобработке данных:

- Датасет `mark_events`: преобразовать значения `start_dt` и `finish_dt` в даты.


- `users` — `first_date` в даты;


- `events`:
  - `event_dt` в дату и время;
  - Добавить столбец с датой без времени.

<div align='right'><a href='#content'>↑ В начало проекта ↑</a></div>

---

## <a id='step-2'>Шаг 2. Предобработка данных</a>

1. Преобразуем даты в столбцах датасетов:

In [ ]:
# список датасетов, в которых есть столбцы с датами
list_of_dfs = (mark_events, users, events)

for df in list_of_dfs:
    for col in df.columns:
        if 'dt' in col or 'date' in col:
            df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
            print(f'Тип данных в столбцe {col}: {df[col].dtypes}')

2. В датафрейме `events` выделим только дату события в отдельный столбец:

In [ ]:
events['event_date'] = pd.to_datetime(events['event_dt'].dt.date)
events.sample(5)

In [ ]:
events.info()

### Промежуточные выводы

На этом этапе мы преобразовали типы данных в столбцах с датами и выделили только дату без времени в датафрейме `events`.

<div align='right'><a href='#content'>↑ В начало проекта ↑</a></div>

---

## <a id='step-3'>Шаг 3. Оценка корректности проведения A/B-теста</a>

1. Зафиксируем в переменных условия теста:

In [ ]:
# название теста
test_name = 'recommender_system_test'

# дата запуска теста
test_start = datetime(2020,12,7).strftime('%Y-%m-%d')

# дата остановки набора новых пользователей
new_users_end = datetime(2020,12,21).strftime('%Y-%m-%d')

# дата остановки
test_end = datetime(2021,1,4).strftime('%Y-%m-%d')

# регион (для переиспользования можно задать в виде списка,
# если в будущих тестах могут участвовать пользователи из нескольких регионов)
region = 'EU'

# процент новых пользователей из целевого региона
au_pcnt = 15

# ожидаемое количество участников теста
users_cnt_expected = 6000

# конверсионное окно в днях
conversion_window = 14

# конверсионные метрики
conversion_metrics = {'product_page':'просмотр карточек товаров',
                      'product_cart':'просмотр корзины',
                      'purchase':'покупки'}

2. Отберем новых пользователей из региона EU, которые зарегистрировались в период с `test_start` по `new_users_end`:

In [ ]:
users_eu = users[(users['region'] == region) \
                 & (users['first_date'] >= test_start) & (users['first_date'] <= new_users_end)]
total_users_eu = users_eu.shape[0]
print(f'Всего за период с {test_start} по {new_users_end} пришло {total_users_eu} новых пользователей из региона {region}.')

3. Присоединим к списку новых пользователей из предыдущего пункта данные по участникам тестов (пересечение по `user_id`):

In [ ]:
tests_eu = users_eu.merge(tests, on=['user_id'], how='inner')

print(f'Количество новых пользователей из {region}, попавших хотя бы в один A/B-тест: {len(tests_eu)}')

In [ ]:
tests_eu.sample(5)

4. Проверим, есть ли пользователи, которые попали сразу в обе группы нашего теста:

In [ ]:
users_in_ab = (tests_eu.query('ab_test == @test_name')
               .groupby(['user_id', 'ab_test']).agg({'group':'nunique'})
               .sort_values(by='group', ascending=False))

print('Количество пользователей, попавших в обе группы теста {}: {}'.format(test_name, len(users_in_ab.query('group > 1'))))

5. Отберем ID пользователей, которые участвовали в нашем тесте:

In [ ]:
test_users_ids = list(tests_eu.query('ab_test == @test_name')['user_id'])
display(test_users_ids[:5])
print('Количество пользователей, участвовавших в тесте {}: {}'.format(test_name, len(test_users_ids)))

6. Проверим, пересекается ли аудитория нашего теста с другими тестами:

In [ ]:
users_in_manytests = (tests_eu.query('user_id in @test_users_ids')
                      .groupby(['user_id'], as_index=False).agg({'ab_test':'nunique'})
                      .query('ab_test > 1'))
users_in_manytests.sample(5)

Видим, что аудитория нашего теста пересекается с другими тестами. Оценим масштаб бедствия:

In [ ]:
# записываем id пользователей, попавших в несколько тестов
users_in_manytests_ids = list(users_in_manytests['user_id'])

users_in_manytests_cnt = len(users_in_manytests_ids)

print('Количество пользователей, попавших в несколько тестов: {:.0f} ({:.1%})'
      .format(users_in_manytests_cnt, users_in_manytests_cnt/len(test_users_ids)))

Видим, что четверть аудитории нашего теста попали и в другой / другие тесты тоже.

Кажется, что **было бы правильно не использовать данные по пользователям, попавшим в несколько тестов**. Но с другой стороны, я думаю, что в реальных продуктах одновременно может идти много тестов, и пересечение неизбежно.

Думаю, что **можно оставить этих пользователей, если в каждой группе нашего теста примерно одинаковая доля участников из разных групп в других тестах**. Например, если в группе A нашего теста 100 человек из группы B другого теста (и это составляет 5% от общего числа пользователей в группе А нашего теста), то среди участников группы B нашего теста должно быть тоже примерно 5% пользователей, попавших в группу B другого теста.

Проверим это:

In [ ]:
# запишем общее количество участников по группам нашего теста
in_other_tests = (tests_eu.query('ab_test == @test_name')
                  .groupby(['group'], as_index=False).agg({'user_id':'nunique'})
                  .rename(columns={'user_id':'users_cnt'}))
display(in_other_tests)

# проверим, совпадает ли сумма по группам
print(in_other_tests['users_cnt'].sum())

In [ ]:
# id пользователей в группе A нашего теста
users_a = list(tests_eu.query('ab_test == @test_name & group=="A"')['user_id'].unique())
len(users_a)

In [ ]:
# id пользователей в группе B нашего теста
users_b = list(tests_eu.query('ab_test == @test_name & group=="B"')['user_id'].unique())
len(users_b)

In [ ]:
# перебираем все другие тесты (если их будет больше двух)
for test in tests_eu['ab_test'].unique():
    
    # исключаем целевой тест, с которым мы работаем    
    if test != test_name:
        work_df = tests_eu[tests_eu['ab_test'] == test]
        
        # перебираем группы другого теста
        for group in work_df['group'].unique():
            # записываем id пользователей, попавших в каждую группу другого теста
            users_in_group = list(work_df.loc[work_df['group'] == group, 'user_id'].unique())
        
            # считаем количество пересечений по id пользователей в группах нашего теста и группе другого теста
            intersection_a_cnt = len(set(users_in_group).intersection(users_a))
            intersection_b_cnt = len(set(users_in_group).intersection(users_b))
        
            # собираем датафрейм
            cnt_name = test + '_' + group
            intersection_cnt = pd.DataFrame({'group':['A', 'B'], cnt_name:[intersection_a_cnt, intersection_b_cnt]})
        
            # присоединяем полученный датафрейм к датафрейму с количеством участников по группам нашего теста
            in_other_tests = in_other_tests.merge(intersection_cnt, on='group', how='left')
        
            # имя колонки с процентами
            pcnt_name = cnt_name + '_pcnt'
            in_other_tests[pcnt_name] = round(in_other_tests[cnt_name] / in_other_tests['users_cnt'] * 100, 2)
        
in_other_tests

Видим, что доли пользователей, которые попали в экспериментальную группу B и контрольную группу А другого теста (`interface_eu_test`), примерно одинаковые в наших группах. Поэтому считаю, что можно оставить этих пользователей (если изменения, тестируемые в другом тесте, повлияли на поведение наших пользователей, то эффект должен быть примерно одинаковым в обеих группах).

7. Проверим равномерность распределения пользователей по группам нашего теста:

In [ ]:
round(abs(min(in_other_tests['users_cnt']) / max(in_other_tests['users_cnt']) - 1) * 100, 1)

Относительное различие в количестве пользователей в группах очень большое – 25%. Это говорит о том, что траффик распределялся неравномерно.

8. Проверим, какая доля от всех новых пользователей из EU за период с `test_start` по `new_users_end`, попала в наш тест (по ТЗ должно быть около 15%):

In [ ]:
round(len(test_users_ids) / total_users_eu * 100, 1)

Тут идеальное попадание в ТЗ.

9. И еще посмотрим, проходили ли какие-то маркетинговые активности в регионе EU во время нашего теста:

In [ ]:
mark_events

Отбираем по региону:

In [ ]:
mark_events['in_eu'] = mark_events['regions'].apply(lambda x: 'EU' in x.split(', '))
mark_events_eu = mark_events.query('in_eu == True')
mark_events_eu

Отбираем по датам:

In [ ]:
print(test_start)
print(test_end)

In [ ]:
mark_events_eu_time = mark_events_eu.query('finish_dt >= @test_start & finish_dt <= @test_end')

if len(mark_events_eu_time) == 0:
    print('Во время теста нет маркетинговых событий.')
else:
    display(mark_events_eu_time)

### Промежуточные выводы

На этом этапе мы проверили корректность проведения теста и отобрали пользователей, данные по которым будем анализировать дальше.

- Всего за период с 2020-12-07 по 2020-12-21 пришло 42340 новых пользователей из региона EU;
- Из этих пользователей попали хотя бы в один тест 16916 человек;
- В интересующем нас тесте `recommender_system_test` участвовали 6351 человек (15% от общего количества новых пользователей из EU = соответствует ТЗ), из них никто не попал в обе группы этого теста;
- 1602 пользователя (25.2%) участвовали в двух тестах одновременно, но доли пользователей, которые попали в экспериментальную группу B и контрольную группу А другого теста, примерно одинаковые в группах A и B нашего теста. Поэтому мы решили оставить этих пользователей;
- Относительное различие в количестве пользователей в группах очень большое – 25%. Это говорит о том, что траффик распределялся неравномерно;
- Во время теста было маркетинговое мероприятие «Christmas&New Year Promo» – это нужно будет учесть при анализе активностей пользователей.


Вердикт по корректности проведения теста:
- Длительность теста при неравномерном распределении должна быть увеличена для обнаружения статистически значимой разницы (если она есть на самом деле), так как минимальная длительности достигается при делении 50/50;
- Проведение новогодней акции могло повлиять на активность пользователей;
- Так же сомнительным является факт участие 25% пользователей в двух тестах одновременно (хотя в ТЗ и нет такого ограничения).

<div align='right'><a href='#content'>↑ В начало проекта ↑</a></div>

---

## <a id='step-4'>Шаг 4. Исследовательский анализ данных</a>

1. Соберем все нужные данные в один датафрейм: отберем пользователей нашего теста, присоединим данные по группам теста, отберем события.

В ТЗ указан ожидаемый эффект: **за 14 дней** с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%. Поэтому при отборе событий оставим для каждого пользователя только события, совершенные за 14 дней с момента регистрации.

Отберем данные по участникам теста:

In [ ]:
users_test = users.query('user_id in @test_users_ids')
display(users_test.sample(5))
len(users_test)

Присоединим данные по группам нашего теста:

In [ ]:
users_groups = users_test.merge(tests.query('ab_test == @test_name'), on='user_id', how='left')

In [ ]:
users_groups.sample(5)

In [ ]:
users_groups['user_id'].nunique()

In [ ]:
groups_cnt = users_groups.groupby(['group'], as_index=False).agg({'user_id':'count'})
groups_cnt

Присоединим данные по событиям:

In [ ]:
users_events = users_groups.merge(events, on='user_id')

In [ ]:
users_events.sample(5)

In [ ]:
total_users_in_test = groups_cnt['user_id'].sum()
users_with_actions = users_events['user_id'].nunique()
users_with_actions_pcnt = users_with_actions / total_users_in_test
print('Из {:.0f} совершили хотя бы одно действие {:.0f} пользователей ({:.1%})'.\
      format(total_users_in_test, users_with_actions, users_with_actions_pcnt))

Посмотрим среднее количество событий на пользователя до отсечения событий после 14 дней:

In [ ]:
users_events.groupby(['user_id']).agg({'user_id':'count'}).mean()

Отберем для каждого пользователя события, совершенные только за первые 14 дней после регистрации (параметр нашего теста `conversion_window`):

In [ ]:
# добавим столбец conversion_date = first_date + conversion_window
users_events['conversion_date'] = users_events['first_date'] + pd.Timedelta(days = conversion_window)
users_events.sample(5)

In [ ]:
len(users_events)

In [ ]:
users_events['group'].value_counts()

In [ ]:
# отберем данные за 14 дней
df = users_events.query('event_date>=first_date & event_date<=conversion_date')
len(df)

In [ ]:
df['user_id'].nunique()

In [ ]:
df.groupby(['user_id']).agg({'user_id':'count'}).mean()

Среднее количество событий на пользователя снизилось с 6.73 до 6.56.

2. Посмотрим, как распределено количество событий на пользователя в выборках:

In [ ]:
df_a = df.query('group == "A"')
df_b = df.query('group == "B"')

In [ ]:
ax = df_a.groupby(['user_id']).agg({'event_name':'count'}).plot(kind='hist', alpha=0.6)
df_b.groupby(['user_id']).agg({'event_name':'count'}).plot(kind='hist', alpha=0.6, ax=ax)

plt.legend(labels=['A','B'])
plt.title('Количество событий на одного пользователя по группам')
plt.xlabel('Количество событий на одного пользователя')
plt.ylabel('Количество пользователей')
plt.show()

In [ ]:
(df.groupby(['group', 'user_id'], as_index=False)['event_name'].agg(['count'])
 .groupby(['group'])['count'].agg(['min','max','mean'])
)

3. Посмотрим, как распределено количество событий по дням:

In [ ]:
ax = df.groupby(['event_date']).agg({'user_id':'count'}).plot()
df_a.groupby(['event_date']).agg({'user_id':'count'}).plot(ax=ax)
df_b.groupby(['event_date']).agg({'user_id':'count'}).plot(ax=ax)

plt.legend(labels=['total','A','B'])
plt.grid()
plt.title('Количество событий по дням')
plt.xlabel('Даты')
plt.ylabel('Количество событий')
plt.show()

In [ ]:
events_per_date = df.groupby(['event_date'], as_index=False).agg({'user_id':'count'})
events_max = events_per_date.max()
events_min = events_per_date.min()

print('Максимум событий произошло {} = {}'.format(events_max[0].strftime(format='%Y-%m-%d'), events_max[1]))
print('Минимум событий произошло {} = {}'.format(events_min[0].strftime(format='%Y-%m-%d'), events_min[1]))

### Промежуточные выводы

На этом этапе выяснили:
- Из 6351 совершили хотя бы одно действие 3481 пользователей (54.8%);
- Характер распределения количества событий на одного пользователя различается: в группе A пик на 8-10, а в группе B – 1-3, но при этом более длинный хвост; среднее в группе А больше – 6.9 против 5.5 в группе B;
- Распределение количества событий по дням в группе B более ровное;
- Максимум событий произошло 2020-12-29 = 2304;
- Минимум событий произошло 2020-12-07 = 332.

 <div align='right'><a href='#content'>↑ В начало проекта ↑</a></div>

---

## <a id='step-5'>Шаг 5. Оценка результатов A/B-тестирования</a>

1. Напишем функцию для проверки гипотез, которая будет:

   - Принимать на вход название события и значение критического уровня статистической значимости (alpha, по умолчанию будет равно 0.05);
   - Выводить анализируемое событие, нулевую и альтернативные гипотезы;
   - Рассчитывать и выводить долю пользователей, совершивших действие, в каждой группе;
   - Проводить z-тест и выводить его результаты.

In [ ]:
def check_hyp(event, alpha=0.05):
    
    # пишем, какое событие анализируем   
    print(colored('Событие {} в группах A и B'.format(event), attrs=['bold']))
    
    # формулируем нулевую и альтернативную гипотезы
    print()
    print('H0: доли пользователей, совершивших событие {}, в группах A и B не различаются.'.format(event))
    print('H1: доли пользователей, совершивших событие {}, в группах A и B различаются.'.format(event))
    
    # считаем количество пользователей, совершивших событие, в каждой группе
    successesA = df_a.loc[df_a['event_name']==event, 'user_id'].nunique()
    successesB = df_b.loc[df_b['event_name']==event, 'user_id'].nunique()
    
    # вытаскиваем общее количество пользователей
    totalA = groups_cnt.loc[groups_cnt['group'] == 'A', 'user_id'].values[0]
    totalB = groups_cnt.loc[groups_cnt['group'] == 'B', 'user_id'].values[0]
    
    # находим доли
    propA = successesA / totalA
    propB = successesB / totalB
    
    # выводим доли + количество пользователей в группах
    print()
    print(f'{propA:.0%} от {totalA} пользователей с событием {event} в группе «A»')
    print(f'{propB:.0%} от {totalB} пользователей с событием {event} в группе «B»')
    
    
    ########## проверяем гипотезу ##########
      
    # пропорция успехов в группах
    pA = successesA / totalA
    pB = successesB / totalB

    # пропорция успехов в комбинированной группе
    p_combined = (successesA + successesB) / (totalA + totalB)

    # разница пропорций в группах
    difference = pA - pB
    
    # выводим, в какой группе конверсия лучше
    if difference > 0:
        print(colored('Конверсия в группе B хуже.', 'red'))
    else:
        print(colored('Конверсия в группе B лучше.', 'greed'))

    # статистика в стандартных отклонениях стандартного нормального распределения
    z_value = difference / mth.sqrt(p_combined * (1 - p_combined) * (1/totalA + 1/totalB))

    # задаем стандартное нормальное распределение (среднее = 0, стандартное отклонение = 1)
    distr = st.norm(0, 1)
    
    # посчитаем, как далеко статистика уехала от нуля = какова вероятность получить такое отличие или больше
    # так как распределение статистики нормальное, вызовем метод cdf()
    # саму z-статистику возьмём по модулю методом abs(), чтобы получить правильный результат независимо от её знака
    # это возможно, потому что тест двусторонний. По этой же причине удваиваем результат:
    p_value = (1 - distr.cdf(abs(z_value))) * 2
    
    print()
    print(f'p-значение: {p_value:.4%}')

    # используем библиотеку termcolor
    if (p_value < alpha):
        print(colored('Отвергаем нулевую гипотезу: между долями есть значимая разница.', 'green'))
    else:
        print(colored('Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными.', 'red'))
    
    print()
    print()
    
    return

2. Так как мы будем проверять три гипотезы на одном наборе данных, используем поправку уровня статистической значимости методом Бонферрони:

In [ ]:
alpha = 0.05
alpha_corr = alpha / 3

3. Проверим статистическую разницу долей по каждому событию:

In [ ]:
for event in ['product_page', 'product_cart', 'purchase']:
    check_hyp(event, alpha=alpha_corr)

### Промежуточные выводы

По всем параметрам группа `B` хуже — конверсия в ней статистически значимо ниже.

<div align='right'><a href='#content'>↑ В начало проекта ↑</a></div>

---

## <a id='result'>Общие выводы</a>

Результаты A/B-теста следует признать неудачными: по всем параметрам группа B хуже — конверсия в ней статистически значимо ниже, чем в контрольной группе A.

При этом нужно заметить, что при проведении теста были особенности:
- 25% пользователей участвовали в двух тестах одновременно;
- Пользователи распределялись по группам неравномерно (относительное различие в количестве пользователей в группах = 25.2%);
- Во время теста проходила новогодняя акция, поэтому поведение пользователей могло быть нестандартным.

В целом, рекомендуем **НЕ внедрять** тестируемое изменение в продукт.

<div align='right'><a href='#content'>↑ В начало проекта ↑</a></div>